In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
from tqdm.auto import tqdm
import random
import datetime
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
import datetime

In [ ]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
train_err2 = train_err.copy()
test_err2 = test_err.copy()

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
# 일단 자료들 날짜형식이나 콤마(,) 제거 등 간단한 전처리 진행

train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()


# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
def quals(dataset, testset):
  data = dataset.copy()
  test = testset.copy()

  cols = ['quality_1', 'quality_2', 'quality_5', 'quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
  a = 0
  for col in tqdm(cols):

    q_list = data[col].unique()
    q_list2= test[col].unique()
    both = [x for x in q_list if x in q_list2]
    both.sort()
    # temp = train_qui2[train_qui2['quality_1'] == ]

    train_idx = data[col].value_counts()
    test_idx = test[col].value_counts()

    fin_list = []
    sum_far = 0

    for bo in both:
      q = train_idx[train_idx.index == bo].values[0]
      qs = test_idx[test_idx.index == bo].values[0]
      if q+qs + sum_far >= 6000:
        fin_list.append(bo)
        sum_far = 0
      elif q+qs + sum_far < 6000:
        sum_far += q+qs

    for i in range(len(fin_list)):
      val = fin_list[i]
      if i == len(fin_list) -1:
        data[col][data[col] > val] = val + 1
        test[col][test[col] > val] = val + 1
      else:
        af_val = fin_list[i+1]
        data[col][(data[col] >= val) & (data[col] < af_val)] = val
        test[col][(test[col] >= val) & (test[col] < af_val)] = val
    print(len(fin_list))
    a += len(fin_list)
  data['date'] = data.time.dt.dayofyear
  test['date'] = test.time.dt.dayofyear

  data['week'] = data.time.dt.weekofyear
  test['week'] = test.time.dt.weekofyear
  print('총 변수의 수 {0}'.format(a))
  return data, test

In [ ]:
train_qui3, test_qui3 = quals(train_qui2, test_qui2)

3
5
32
5
18
6
7
81
2
3

총 변수의 수 162


In [165]:
temp = train_qui3[train_qui3.user_id == 10000]
temp['quality_1']

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
Name: quality_1, dtype: float64

In [205]:
ts = test_qui3.drop(['time', 'date', 'week'], axis = 1)
ts.set_index(keys = 'user_id', inplace = True)

In [206]:
k = ts.astype('str')

In [207]:
kk = pd.get_dummies(k)

In [208]:
a = kk.groupby(by = ['user_id']).sum()
b = kk.groupby(by = ['user_id']).std()
c = kk.groupby(by = ['user_id']).mean()

In [209]:
a.to_csv("테스트_퀼리티_합.csv", index = True)
!cp 테스트_퀼리티_합.csv "drive/My Drive/"
b.to_csv("테스트_퀼리티_분산.csv", index = True)
!cp 테스트_퀼리티_분산.csv "drive/My Drive/"
c.to_csv("테스트_퀼리티_평균.csv", index = True)
!cp 테스트_퀼리티_평균.csv "drive/My Drive/"

In [ ]:
#def qu_stats(dataset, testset):
data = dataset.copy()
test = testset.copy()

cols = ['quality_1', 'quality_2'] #, 'quality_5', 'quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
# ids = data.user_id.unique()
# ids_t = test.user_id.unique()

ids = [10000,10002]
ids_t = [30000,30002]

for col in cols:
  q_list = data[col].unique()
  q_list2= test[col].unique()
  both = [x for x in q_list if x in q_list2]
  both.sort()

  for id in tqdm(ids):
    t_list = []
    temp = data[data.user_id == id]
    for bo in both:
      k = temp[col]


In [ ]:
begins = datetime.date(2020, 10, 31).timetuple().tm_yday
ends = datetime.date(2020, 11, 30).timetuple().tm_yday

In [138]:
def day_maker(trainset, testset):
  data = trainset.copy()
  test = testset.copy()

  ids = data.user_id.unique()
  ids_t = test.user_id.unique()

  # ids = [10000,10002]
  # ids_t = [30000,30002]

  ids.sort()
  ids_t.sort()
  df = pd.DataFrame()
  df_t = pd.DataFrame()

  tr_list = []
  ts_list = []

  for id in tqdm(ids):
    t_list = []
    temp = data[data.user_id == id]
    for i in range(begins, ends + 1):
      k = len(temp[temp['date'] == i])
      t_list.append(k)
    a = sum(t_list)
    b = np.std(t_list)
    c = np.mean(t_list)
    t_list.extend([a,b,c])
    tr_list.append(t_list)

  for id in tqdm(ids_t):
    t_list = []
    temp = test[test.user_id == id]
    for i in range(begins, ends + 1):
      k = len(temp[temp['date'] == i])
      t_list.append(k)
    a = sum(t_list)
    b = np.std(t_list)
    c = np.mean(t_list)
    t_list.extend([a,b,c])
    ts_list.append(t_list)

  train = pd.DataFrame(tr_list, index = ids)
  test = pd.DataFrame(ts_list, index = ids_t)

  return train, test

In [129]:
### 일자별 에러 횟수, 누적, 평균, 분산

train_qui_day, test_qui_day = day_maker(train_qui3, test_qui3)

In [222]:
train_qui_day.to_csv("트레인_퀼리티_일일.csv", index = True)
!cp 트레인_퀼리티_일일.csv "drive/My Drive/"

test_qui_day.to_csv("테스트_퀼리티_일일.csv", index = True)
!cp 테스트_퀼리티_일일.csv "drive/My Drive/"

In [220]:
train_qui_day.columns = names
test_qui_day.columns = names

In [219]:
names = ['qq_days_' + str(x)  for x in range(34)]

In [221]:
train_qui_day

,qq_days_0,qq_days_1,qq_days_2,qq_days_3,qq_days_4,qq_days_5,qq_days_6,qq_days_7,qq_days_8,qq_days_9,qq_days_10,qq_days_11,qq_days_12,qq_days_13,qq_days_14,qq_days_15,qq_days_16,qq_days_17,qq_days_18,qq_days_19,qq_days_20,qq_days_21,qq_days_22,qq_days_23,qq_days_24,qq_days_25,qq_days_26,qq_days_27,qq_days_28,qq_days_29,qq_days_30,qq_days_31,qq_days_32,qq_days_33
10000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,12,24,2.948041,0.774194
10002,0,0,0,0,12,0,12,0,0,0,0,12,0,0,0,12,0,0,12,12,0,0,0,0,0,0,0,0,0,0,24,96,6.071376,3.096774
10004,0,0,12,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,2.948041,0.774194
10005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,24,4.240433,0.774194
10006,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,12,0,0,0,36,3.547801,1.161290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,24,2.948041,0.774194
24992,0,0,0,0,0,0,0,0,24,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,4.677308,1.161290
24993,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,24,2.948041,0.774194
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,24,2.948041,0.774194


In [137]:
def week_maker(trainset, testset):
  data = trainset.copy()
  test = testset.copy()

  ids = data.user_id.unique()
  ids_t = test.user_id.unique()

  # ids = [10000,10002]
  # ids_t = [30000,30002]
  begins = 44
  ends = 49

  ids.sort()
  ids_t.sort()
  df = pd.DataFrame()
  df_t = pd.DataFrame()

  tr_list = []
  ts_list = []

  for id in tqdm(ids):
    t_list = []
    temp = data[data.user_id == id]
    for i in range(begins, ends + 1):
      k = len(temp[temp['week'] == i])
      t_list.append(k)
    a = sum(t_list)
    b = np.std(t_list)
    c = np.mean(t_list)
    t_list.extend([a,b,c])
    tr_list.append(t_list)

  for id in tqdm(ids_t):
    t_list = []
    temp = test[test.user_id == id]
    for i in range(begins, ends + 1):
      k = len(temp[temp['week'] == i])
      t_list.append(k)
    a = sum(t_list)
    b = np.std(t_list)
    c = np.mean(t_list)
    t_list.extend([a,b,c])
    ts_list.append(t_list)

  train = pd.DataFrame(tr_list, index = ids)
  test = pd.DataFrame(ts_list, index = ids_t)

  return train, test

In [139]:
train_qui_week, test_qui_week = week_maker(train_qui3, test_qui3)

In [225]:
train_qui_week.rename(columns = lambda x: 'q_week_'+str(x), inplace = True)
test_qui_week.rename(columns = lambda x: 'q_week_'+str(x), inplace = True)

In [227]:
train_qui_week.to_csv("트레인_퀼리티_주간.csv", index = True)
!cp 트레인_퀼리티_주간.csv "drive/My Drive/"


test_qui_week.to_csv("테스트_퀼리티_주간.csv", index = True)
!cp 테스트_퀼리티_주간.csv "drive/My Drive/"

In [144]:
temp = train_qui3[train_qui3['user_id'] == 10002]

In [150]:
temp['quality_1'][temp['quality_1'] != 0]

2

In [160]:
def not_zero(dataset, testset):
  data = dataset.copy()
  test = testset.copy()

  cols = ['quality_1', 'quality_2', 'quality_5', 'quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
  ids = data.user_id.unique()
  ids_t = test.user_id.unique()

  # ids = [10000,10002]
  # ids_t = [30000,30002]

  tr_list = []
  for id in tqdm(ids):
    temp = data[data['user_id'] == id]
    q_list = []
    for col in cols:
      k = temp[col][temp[col] != 0] + 2
      a = len(k)
      b = k.mean()
      c =k.std()
      q_list.extend([a,b,c])
    tr_list.append(q_list)

  ts_list = []
  for id in tqdm(ids_t):
    temp = test[test['user_id'] == id]
    q_list = []
    for col in cols:
      k = temp[col][temp[col] != 0] + 2
      a = len(k)
      b = k.mean()
      c =k.std()
      q_list.extend([a,b,c])
    ts_list.append(q_list)

  df = pd.DataFrame(tr_list, index = ids)
  dft = pd.DataFrame(ts_list, index = ids_t)

  df = df.fillna(-3)
  dft = dft.fillna(-3)
  return df, dft

In [161]:
train_zero, test_zero = not_zero(train_qui3, test_qui3)

In [203]:
train_zero.to_csv("트레인_퀼리티_0말고.csv", index = True)
!cp 트레인_퀼리티_0말고.csv "drive/My Drive/"


test_zero.to_csv("테스트_퀼리티_0말고.csv", index = True)
!cp 테스트_퀼리티_0말고.csv "drive/My Drive/"

In [159]:
train_zero

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
10000,0,-3.0,-3.0,0,-3.000000,-3.000000,2,6.000000,2.828427,0,-3.0,-3.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,6.000000,2.043016,0,-3.0,-3.0,0,-3.0,-3.0
10002,2,-1.0,0.0,3,-0.333333,1.154701,14,2.071429,1.979288,3,7.0,13.856406,24,23.0,21.451665,0,-3.0,-3.0,12,1.0,0.0,84,4.428571,1.928141,2,-1.0,0.0,0,-3.0,-3.0
10004,2,-1.0,0.0,2,-1.000000,0.000000,5,0.400000,1.341641,3,7.0,13.856406,12,71.0,0.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,2.000000,1.021508,2,-1.0,0.0,0,-3.0,-3.0
10005,10,-1.0,0.0,10,-1.000000,0.000000,16,0.000000,1.460593,12,1.5,6.987001,12,34.0,0.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,5.000000,1.021508,10,-1.0,0.0,0,-3.0,-3.0
10006,0,-3.0,-3.0,0,-3.000000,-3.000000,3,4.000000,0.000000,0,-3.0,-3.000000,12,4.0,0.000000,0,-3.0,-3.0,0,-3.0,-3.0,36,4.000000,0.000000,0,-3.0,-3.0,0,-3.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,8,-1.0,0.0,8,-1.000000,0.000000,10,-0.600000,0.843274,8,-1.0,0.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,1.000000,0.000000,8,-1.0,0.0,0,-3.0,-3.0
24992,12,-1.0,0.0,12,-1.000000,0.000000,16,0.437500,2.851169,12,-1.0,0.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,36,6.333333,2.908117,12,-1.0,0.0,0,-3.0,-3.0
24993,7,-1.0,0.0,7,-1.000000,0.000000,9,0.111111,2.260777,7,-1.0,0.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,4.000000,1.021508,7,-1.0,0.0,0,-3.0,-3.0
24995,10,-1.0,0.0,10,-1.000000,0.000000,12,-0.416667,1.378954,10,-1.0,0.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,2.500000,0.510754,10,-1.0,0.0,0,-3.0,-3.0


In [162]:
train_zero

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
10000,0,-3.0,-3.0,0,-3.000000,-3.000000,2,8.000000,2.828427,0,-3.0,-3.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,8.000000,2.043016,0,-3.0,-3.0,0,-3.0,-3.0
10002,2,1.0,0.0,3,1.666667,1.154701,14,4.071429,1.979288,3,9.0,13.856406,24,25.0,21.451665,0,-3.0,-3.0,12,3.0,0.0,84,6.428571,1.928141,2,1.0,0.0,0,-3.0,-3.0
10004,2,1.0,0.0,2,1.000000,0.000000,5,2.400000,1.341641,3,9.0,13.856406,12,73.0,0.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,4.000000,1.021508,2,1.0,0.0,0,-3.0,-3.0
10005,10,1.0,0.0,10,1.000000,0.000000,16,2.000000,1.460593,12,3.5,6.987001,12,36.0,0.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,7.000000,1.021508,10,1.0,0.0,0,-3.0,-3.0
10006,0,-3.0,-3.0,0,-3.000000,-3.000000,3,6.000000,0.000000,0,-3.0,-3.000000,12,6.0,0.000000,0,-3.0,-3.0,0,-3.0,-3.0,36,6.000000,0.000000,0,-3.0,-3.0,0,-3.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,8,1.0,0.0,8,1.000000,0.000000,10,1.400000,0.843274,8,1.0,0.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,3.000000,0.000000,8,1.0,0.0,0,-3.0,-3.0
24992,12,1.0,0.0,12,1.000000,0.000000,16,2.437500,2.851169,12,1.0,0.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,36,8.333333,2.908117,12,1.0,0.0,0,-3.0,-3.0
24993,7,1.0,0.0,7,1.000000,0.000000,9,2.111111,2.260777,7,1.0,0.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,6.000000,1.021508,7,1.0,0.0,0,-3.0,-3.0
24995,10,1.0,0.0,10,1.000000,0.000000,12,1.583333,1.378954,10,1.0,0.000000,0,-3.0,-3.000000,0,-3.0,-3.0,0,-3.0,-3.0,24,4.500000,0.510754,10,1.0,0.0,0,-3.0,-3.0
